In [1]:
import moldesign as mdt
from moldesign import units as u

OpenMM could not be imported; using remote docker container
PDBFixer could not be imported; using remote docker container
PySCF not installed; using remote docker container
Reading configuration from /Users/t_leeday/.moldesign/moldesign.yml


In [2]:
from moldesign.models.lammps_model import LAMMPSPotential
import parmed as med

In [3]:
mol = mdt.from_pdb('1yu8')


In [4]:
mdmol = mdt.assign_forcefield(mol)

print "Done parameterizing"



Connecting to docker host at unix://var/run/docker.sock... done


Done parameterizing


In [5]:
mdmol.set_energy_model(LAMMPSPotential)
print "Done setting energy model"


Done setting energy model


In [6]:
hello = mdmol.calculate()
print "Done calculating"

LAMMPS output is captured by PyLammps wrapper
Done calculating


In [7]:
hello['potential_energy']

<Quantity(-4179.44402414, 'kcalpermol')>

In [8]:
mdmol.set_integrator(mdt.integrators.LAMMPSNvt,
                   timestep=1.0*u.fs,
                   temperature=300.0*u.kelvin,
                   frame_interval=10.0*u.fs)

In [9]:
print mdmol.atoms[1004].position

[ 11.254   7.626  12.101] ang


In [10]:
mdmol.positions[0].value_in(u.angstrom)

array([ 10.367,  29.903,  18.753])

In [11]:
traj = mdmol.run(100*u.fs)

LAMMPS output is captured by PyLammps wrapper
Done - integrated "Molecule: Unnamed molecule from OpenMM" from 0.0 fs to 100.0 fs


In [12]:
print mdmol.masses

[ 14.00672    1.007947   1.007947 ...,  15.99943    1.007947   1.007947] amu


In [13]:
print mdmol.atoms[1004].position
print mdmol.atoms[1004].velocity

[ 11.2176    7.62717  12.192  ] ang
[-0.0018164  -0.00129114  0.0022006 ] ang / fs


In [14]:
traj.draw()